In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn,optim
import torch.nn.functional as F
import datetime
from torch.utils import data
from random import sample,seed,shuffle


In [2]:
flights = pd.read_csv("./lessThan1.csv",low_memory=False)
airlines = pd.read_csv("./data/airlines.csv")
airports = pd.read_csv("./data/airports.csv")

In [3]:
a = flights.ORIGIN_AIRPORT.unique()
for i in flights.DESTINATION_AIRPORT.unique():
    if i not in a:
        a = np.append(a,i)
airports = {ch:i for i,ch in enumerate(a)}
airlines = {ch:i for i,ch in enumerate(airlines.IATA_CODE)}
tail_nums = {ch:i for i,ch in enumerate(flights.TAIL_NUMBER.unique())}
airports['10268'] =len(airports)

def one_hot_encode(size,val):
    a = np.zeros((size,),dtype=int)
    a[(val-1)] = 1
    return a
flights['DATE'] = pd.to_datetime(flights[['YEAR','MONTH', 'DAY']])
variables_to_remove = ['TAXI_OUT', 'TAXI_IN', 'WHEELS_ON', 'WHEELS_OFF', 'YEAR', 
                       'DATE', 'AIR_SYSTEM_DELAY',
                       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
                       'WEATHER_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
                       'FLIGHT_NUMBER', 'AIR_TIME']
flights.drop(variables_to_remove, axis = 1, inplace = True)
flights = flights[['AIRLINE','TAIL_NUMBER','DAY', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
        'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
        'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY',
        'SCHEDULED_TIME', 'ELAPSED_TIME','MONTH']]
flights['AIRLINE'] = flights['AIRLINE'].apply(lambda x: airlines[x])
flights['ORIGIN_AIRPORT_V'] = flights['ORIGIN_AIRPORT'].apply(lambda x: airports[x])
flights['DESTINATION__AIRPORT_V'] = flights['DESTINATION_AIRPORT'].apply(lambda x: airports[x])
flights['TAIL_NUMBER'] = flights['TAIL_NUMBER'].apply(lambda x: tail_nums[x])
# flights['DAY_OF_WEEK'] = flights['DAY_OF_WEEK'].apply(lambda x: one_hot_encode(7,x))
flights['DAY'] = flights['DAY'].apply(lambda x: one_hot_encode(31,x))
flights['MONTH'] = flights['MONTH'].apply(lambda x: one_hot_encode(12,x))
flights = flights[['AIRLINE','TAIL_NUMBER','ORIGIN_AIRPORT_V',"DESTINATION__AIRPORT_V",'ARRIVAL_DELAY','DEPARTURE_DELAY','DAY','MONTH','SCHEDULED_ARRIVAL']]
flights.dropna(inplace =True)
flights = flights.reset_index()
# flights = flights[['AIRLINE','ORGIN_AIRPORT_V',"DESTINATION__AIRPORT_V",'ARRIVAL_DELAY','DAY_OF_WEEK','DEPARTURE_DELAY','DAY','MONTH','SCHEDULED_ARRIVAL']]
flights.head()

,index,AIRLINE,TAIL_NUMBER,ORIGIN_AIRPORT_V,DESTINATION__AIRPORT_V,ARRIVAL_DELAY,DEPARTURE_DELAY,DAY,MONTH,SCHEDULED_ARRIVAL
0,0,7,0,0,23,25,12,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",542
1,1,7,1,1,15,43,72,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",549
2,2,11,2,2,51,15,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",555
3,3,4,3,3,42,20,-3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",500
4,4,2,4,4,1,60,60,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",850


In [4]:
flights.columns

Index(['index', 'AIRLINE', 'TAIL_NUMBER', 'ORIGIN_AIRPORT_V',
       'DESTINATION__AIRPORT_V', 'ARRIVAL_DELAY', 'DEPARTURE_DELAY', 'DAY',
       'MONTH', 'SCHEDULED_ARRIVAL'],
      dtype='object')

In [5]:
# '10268' in flights.DESTINATION_AIRPORT.unique()

In [6]:
# len(flights.DESTINATION_AIRPORT.unique())

In [7]:
# a = flights.ORIGIN_AIRPORT.unique()
# print(len(a))
# for i in flights.DESTINATION_AIRPORT.unique():
#     if i not in a:
#         a = np.append(a,i)
        
# len(a)

In [8]:
class Generator(data.Dataset):
    def __init__(self,indexes):
        self.indexes = indexes
    def __len__(self):
        return len(self.indexes)
    def __getitem__(self,index):
        index = self.indexes[index]
        y = flights['ARRIVAL_DELAY'][index]
        y = torch.tensor(y,dtype=torch.float)
#         avg_index = self.flights[(flights['AIRLINE'] == self.flights['AIRLINE'][index])&(flights['ORGIN_AIRPORT_V'] == self.flights['ORGIN_AIRPORT_V'][index])&(flights['DESTINATION__AIRPORT_V'] == self.flights['DESTINATION__AIRPORT_V'][index])]['ARRIVAL_DELAY'][:index].to_list()
#         avg_index_index = sum(avg_index)/len(avg_index) if len(avg_index) > 0 else 0
        x = {
            'TAIL_NUMBER':torch.tensor(flights['TAIL_NUMBER'][index],dtype=torch.float),
            'AIRLINE': torch.tensor(flights['AIRLINE'][index],dtype=torch.long),
            'DAY': torch.tensor(flights['DAY'][index],dtype=torch.float),
            'MONTH': torch.tensor(flights['MONTH'][index],dtype=torch.float),
            'SCHEDULED_ARRIVAL': torch.tensor(flights['SCHEDULED_ARRIVAL'][index],dtype=torch.float),
            'ORIGIN_AIRPORT': torch.tensor(flights['ORIGIN_AIRPORT_V'][index],dtype=torch.long),
            'DESTINATION__AIRPORT': torch.tensor(flights['DESTINATION__AIRPORT_V'][index],dtype=torch.long),
#             'AVERAGE_DELAY': torch.tensor(avg_index,dtype=torch.float)
        }
        return x,y

In [9]:
len(flights)

744347

In [10]:
seed(1234)
indexes = list(flights.index)
shuffle(indexes)
train_indexes = indexes[:600000]
validation_indexes = indexes[600000:650000]
test_indexes = indexes[650000:]
# train_index = sample(indexes,round(len(indexes)*0.75))

In [11]:
params = {'batch_size': 256,
          'shuffle': False,
          'num_workers': 6}

In [12]:
# train = flights.loc[train_index,:]

In [13]:
train_gen = Generator(train_indexes)
train_loader = data.DataLoader(train_gen,**params)
validatation_gen = Generator(validation_indexes)
validatation_loader = data.DataLoader(validatation_gen,**params)

In [14]:
train_gen.__getitem__(0)

({'TAIL_NUMBER': tensor(457.),
  'AIRLINE': tensor(3),
  'DAY': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  'MONTH': tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
  'SCHEDULED_ARRIVAL': tensor(1830.),
  'ORIGIN_AIRPORT': tensor(10),
  'DESTINATION__AIRPORT': tensor(67)},
 tensor(16.))

In [15]:
# train_gen.flights['ARRIVAL_DELAY'][0]

In [16]:
len(airports.keys())

627

In [17]:
flights.DESTINATION__AIRPORT_V.nunique()

625

In [18]:
flights.columns

Index(['index', 'AIRLINE', 'TAIL_NUMBER', 'ORIGIN_AIRPORT_V',
       'DESTINATION__AIRPORT_V', 'ARRIVAL_DELAY', 'DEPARTURE_DELAY', 'DAY',
       'MONTH', 'SCHEDULED_ARRIVAL'],
      dtype='object')

In [19]:
class Model(nn.Module):
    def __init__(self,drop=0.25,airport_len=len(airports.keys())+1,airline_len=len(airlines.keys())):
        super(Model,self).__init__()
        self.input = 45+30+30+15#+1
        self.drop_p = drop
        self.airport = nn.Embedding(airport_len,30)
        self.airline = nn.Embedding(airline_len,15)
        self.layer1 = nn.Linear(self.input,1024)
        self.layer2 = nn.Linear(1024,1024)
        self.layer3 = nn.Linear(1024,1024)
        self.layer4 = nn.Linear(1024,1)
        self.dropout = nn.Dropout(p=drop)
    
    def forward(self,x):
        if torch.cuda.is_available():
            x['TAIL_NUMBER'] = x['TAIL_NUMBER'].cuda()
            x['AIRLINE'] = x['AIRLINE'].cuda()
            x['SCHEDULED_ARRIVAL'] = x['SCHEDULED_ARRIVAL'].cuda()
            x['ORIGIN_AIRPORT'] = x['ORIGIN_AIRPORT'].cuda()
            x['DESTINATION__AIRPORT'] = x['DESTINATION__AIRPORT'].cuda()
            x['DAY'] = x['DAY'].cuda()
            x['MONTH'] = x['MONTH'].cuda()
        batch_size = x['DAY'].shape[0]
        x['TAIL_NUMBER'] = x['TAIL_NUMBER'].reshape(batch_size,-1)
#         x['AVERAGE_DELAY'] = x['AVERAGE_DELAY'].reshape(batch_size,-1)
        x['AIRLINE'] = self.airline(x['AIRLINE']) 
        x['AIRLINE'] = x['AIRLINE'].reshape(batch_size,-1)
        x['SCHEDULED_ARRIVAL'] = x['SCHEDULED_ARRIVAL'].reshape(batch_size,-1)
        x['ORIGIN_AIRPORT'] = self.airport(x['ORIGIN_AIRPORT'])
        x['DESTINATION__AIRPORT'] = self.airport(x['DESTINATION__AIRPORT'])
        x['ORIGIN_AIRPORT'] = x['ORIGIN_AIRPORT'].reshape(batch_size,-1)
        x['DESTINATION__AIRPORT'] = x['DESTINATION__AIRPORT'].reshape(batch_size,-1)
        inp = torch.cat((x['DESTINATION__AIRPORT'],x['ORIGIN_AIRPORT'],x['SCHEDULED_ARRIVAL'],x['AIRLINE'],x['TAIL_NUMBER'],
                        x['DAY'],x['MONTH']),dim=1)
#         print(inp.shape)
        out = self.layer1(inp)
#         print(out.shape)
        out = torch.relu(out)
#         print(out.shape)
        out = self.dropout(out)
        out = self.layer2(out)
        out = torch.relu(out)
        out = self.dropout(out)
#         out = F.dropout(F.relu(out),p=self.drop_p)
        out = self.layer3(out)
        out = torch.relu(out)
        out = self.dropout(out)
#         out = F.dropout(F.relu(out),p=self.drop_p)
        out = self.layer4(out)
        out = out.flatten()
        return out

In [20]:
model = Model()
lr = 0.0001
criterian = nn.SmoothL1Loss() 
optimizer = optim.Adam(model.parameters(),lr =lr)

In [21]:
epoch = 30
if torch.cuda.is_available():
    model.cuda()
for e in range(epoch):
    run_loss =0
    count = 0
    model.train()
    for x,y in train_loader:
        optimizer.zero_grad()
#         torch.cuda.empty_cache()
#       moving to cuda
        if torch.cuda.is_available():
            y = y.cuda()
#         y = y.reshape(y.shape[0],-1)
        out = model(x)
        loss = criterian(out,y)
        run_loss += loss.item()
        loss.backward()
        optimizer.step()
        count +=1
        torch.cuda.empty_cache()
        del x
        del y
    else:
        run_loss = run_loss/count
        model.eval()
        val_loss =0 
        loss = 0
        c =0 
        for x,y in validatation_loader:
            if torch.cuda.is_available():
                y = y.cuda()
#             y = y.reshape(y.shape[0],-1)
            out = model(x)
            loss = criterian(out,y)
            val_loss += loss.item()
            c +=1
        val_loss = val_loss/c
        print('{} loss = {} and validation loss = {}'.format(e,run_loss,val_loss))

0 loss = 11.091658576356673 and validation loss = 12.711618910030444
1 loss = 10.796484482166303 and validation loss = 12.55972891437764
2 loss = 10.373108798733343 and validation loss = 12.199917910050372
3 loss = 10.138260925589162 and validation loss = 11.468391491442311
4 loss = 10.030290351385956 and validation loss = 11.197020924821192
5 loss = 9.9585501346165 and validation loss = 10.949489890312663
6 loss = 9.905977290644987 and validation loss = 10.9036117135262
7 loss = 9.868909358164556 and validation loss = 10.947740335853732
8 loss = 9.84914677094274 and validation loss = 10.938421750555234
9 loss = 9.83712900742736 and validation loss = 10.900386031793088
10 loss = 9.823949481846936 and validation loss = 11.007087644265622
11 loss = 9.81599018354058 and validation loss = 10.98270516979451
12 loss = 9.805630698138943 and validation loss = 10.926818268639702
13 loss = 9.797151847504106 and validation loss = 10.732805714315298
14 loss = 9.79141088842125 and validation loss =

In [22]:
torch.cuda.is_available()

True

In [23]:
len(flights)

744347

In [24]:
len(flights)/1024

726.9013671875

In [25]:
test_gen = Generator(test_indexes)
test_loader = data.DataLoader(test_gen,**params)

In [26]:
x,y = next(iter(test_loader))
model.eval()
# if torch.cuda.is_available():
#             y = y.cuda()
out = model(x)
print('{} is out and actual {}'.format(out,y))

tensor([24.3524, 24.8214, 25.3912, 26.2980, 25.5124, 26.2657, 26.9563, 24.5437,
        24.4749, 24.6806, 25.0022, 25.6633, 23.3554, 25.9532, 26.1658, 24.6174,
        23.1639, 25.3550, 23.7485, 24.7047, 23.5513, 24.4999, 24.3508, 23.5976,
        25.7024, 26.3938, 24.1144, 24.6919, 25.3902, 24.7845, 22.0899, 22.0481,
        24.6121, 24.1167, 25.4296, 25.1309, 25.1832, 25.2505, 25.6115, 25.1099,
        23.4570, 24.6355, 25.0367, 24.2648, 24.9083, 26.1645, 22.0093, 24.9303,
        24.7508, 24.6097, 24.0396, 24.7168, 23.5333, 25.7687, 24.9598, 24.2165,
        24.7652, 26.9681, 25.5013, 26.8429, 25.4674, 26.4425, 24.4637, 25.2804,
        23.9779, 26.1427, 25.0803, 25.1812, 25.8170, 24.2220, 24.2149, 25.6366,
        25.2094, 25.9936, 25.1883, 25.2137, 25.3795, 23.5727, 24.3712, 25.4681,
        26.0440, 25.6656, 24.5508, 24.8185, 24.6940, 26.0496, 24.0744, 25.7086,
        23.6399, 26.3130, 26.0699, 26.0104, 26.6724, 27.0148, 24.8306, 25.7012,
        25.6456, 25.0686, 24.4065, 23.76

In [27]:
out.shape

torch.Size([256])

In [28]:
out = out.flatten()
out.shape

torch.Size([256])

In [29]:
for i in range(len(y)):
    print('{} is out and actual {}'.format(out[i],y[i]))

24.35236167907715 is out and actual 40.0
24.821392059326172 is out and actual 25.0
25.391191482543945 is out and actual 22.0
26.297962188720703 is out and actual 55.0
25.512357711791992 is out and actual 17.0
26.26569938659668 is out and actual 16.0
26.95625114440918 is out and actual 55.0
24.543703079223633 is out and actual 30.0
24.474891662597656 is out and actual 38.0
24.680578231811523 is out and actual 17.0
25.002206802368164 is out and actual 33.0
25.663305282592773 is out and actual 47.0
23.35542106628418 is out and actual 17.0
25.953195571899414 is out and actual 21.0
26.165786743164062 is out and actual 26.0
24.61738395690918 is out and actual 45.0
23.163930892944336 is out and actual 54.0
25.354991912841797 is out and actual 23.0
23.748519897460938 is out and actual 22.0
24.70465850830078 is out and actual 39.0
23.5512638092041 is out and actual 15.0
24.499895095825195 is out and actual 34.0
24.35078239440918 is out and actual 17.0
23.59764289855957 is out and actual 45.0
25

In [30]:
torch.cuda.empty_cache()

In [31]:
c =0 
for i in range(len(y)):
    if abs(y[i]-out[i]) <10:
        c+=1
c

161

In [32]:
c/len(y)

0.62890625

In [34]:
c1=0
c=0
for x,y in test_loader:
            if torch.cuda.is_available():
                y = y.cuda()
#             y = y.reshape(y.shape[0],-1)
            out = model(x)
            loss = criterian(out,y)
            val_loss += loss.item()
            for i in range(len(y)):
                c+=1
                if abs(y[i]-out[i]) <10:
                    c1+=1

In [35]:
c1/c

0.6444614031182762

In [38]:
val_loss/len(test_loader)

9.917766816254362